# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [1]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [2]:
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd
import warnings

from itertools import combinations
from copy import deepcopy
from collections import Counter

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [3]:
def build_classifiers():
    models = [LinearRegression(), KNeighborsClassifier(), SVC(), 
          DecisionTreeClassifier(), GaussianNB(), QuadraticDiscriminantAnalysis()]

    return [model.fit(data_set, labels) for model in models]

In [4]:
def build_stacked_classifier(classifiers, meta_classifier):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = meta_classifier
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [5]:
warnings.filterwarnings("ignore")

classifiers = build_classifiers()
data = []

index_combinations = list(combinations(range(len(classifiers)), 3))

classifier_combinations = [tuple(classifiers[i] for i in combo) for combo in index_combinations]

for combination in classifier_combinations:
    for classifier in classifiers:
        if not isinstance(classifier, LinearRegression):
            meta_classifier = deepcopy(classifier)
            predicted = build_stacked_classifier(combination, meta_classifier)
            accuracy = accuracy_score(test_labels, predicted)
            
            data.append((*combination, meta_classifier, accuracy))
            
data.sort(key=lambda x:x[-1], reverse=True)
df = pd.DataFrame(data, columns=["Classifier 1", "Classifier 2", "Classifier 3", "Meta Classifier", "Accuracy"])
df.head(20)

,Classifier 1,Classifier 2,Classifier 3,Meta Classifier,Accuracy
0,LinearRegression(),KNeighborsClassifier(),SVC(),GaussianNB(),1.0
1,LinearRegression(),KNeighborsClassifier(),DecisionTreeClassifier(),GaussianNB(),1.0
2,LinearRegression(),KNeighborsClassifier(),GaussianNB(),GaussianNB(),1.0
3,LinearRegression(),KNeighborsClassifier(),QuadraticDiscriminantAnalysis(),GaussianNB(),1.0
4,LinearRegression(),SVC(),DecisionTreeClassifier(),GaussianNB(),1.0
5,LinearRegression(),SVC(),GaussianNB(),GaussianNB(),1.0
6,LinearRegression(),SVC(),QuadraticDiscriminantAnalysis(),GaussianNB(),1.0
7,LinearRegression(),DecisionTreeClassifier(),GaussianNB(),GaussianNB(),1.0
8,LinearRegression(),DecisionTreeClassifier(),QuadraticDiscriminantAnalysis(),GaussianNB(),1.0
9,LinearRegression(),GaussianNB(),QuadraticDiscriminantAnalysis(),GaussianNB(),1.0


## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [6]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

Fill the two functions below:

In [7]:
def set_new_weights(model):
    I = np.array((model.predict(test_set) == test_labels).astype(int))
    weights = (1 + I) / (1 + I).sum()
    return weights

Train the classifier with the code below:

In [8]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)

[0.00131406 0.00065703 0.00131406 0.00065703 0.00065703 0.00131406
 0.00131406 0.00131406 0.00131406 0.00065703 0.00131406 0.00065703
 0.00131406 0.00065703 0.00065703 0.00131406 0.00065703 0.00065703
 0.00065703 0.00131406 0.00065703 0.00131406 0.00065703 0.00131406
 0.00065703 0.00065703 0.00065703 0.00065703 0.00131406 0.00131406
 0.00065703 0.00131406 0.00065703 0.00131406 0.00131406 0.00131406
 0.00131406 0.00065703 0.00065703 0.00065703 0.00131406 0.00065703
 0.00065703 0.00065703 0.00131406 0.00065703 0.00065703 0.00131406
 0.00131406 0.00131406 0.00065703 0.00131406 0.00065703 0.00131406
 0.00065703 0.00065703 0.00065703 0.00131406 0.00065703 0.00131406
 0.00065703 0.00131406 0.00065703 0.00065703 0.00065703 0.00065703
 0.00131406 0.00065703 0.00131406 0.00131406 0.00131406 0.00065703
 0.00065703 0.00131406 0.00065703 0.00065703 0.00065703 0.00065703
 0.00065703 0.00131406 0.00131406 0.00131406 0.00065703 0.00065703
 0.00065703 0.00131406 0.00065703 0.00131406 0.00065703 0.0013

Set the validation data set:

In [9]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [10]:
def get_prediction(x):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(x))
    output = np.array(output)
    predicted = []
    for i in range(len(x)):
        classified = output[:, i]
        counts = np.bincount(classified)
        predicted.append(np.argmax(counts))
    return predicted

Test it:

In [11]:
prediction = get_prediction(validate_x)[0]

print(prediction)

1
